In [1]:
import numpy as np
import pandas as pd

In [ ]:
def split_dataset(filepath="./Boston-filtered.csv", testsize=1/3, random_state=None):
    """
    Split a dataset into training and test sets.

    This function loads a dataset from a CSV file, shuffles the rows (if a random seed is provided), 
    and splits the dataset into input features (`X`) and target values (`y`). The dataset is then 
    divided into training and test sets based on the specified `testsize` ratio.

    Parameters:
    ----------
    filepath : str, optional
        The path to the CSV file containing the dataset. The default is "./Boston-filtered.csv".
    
    testsize : float, optional
        The proportion of the dataset to include in the test split. Default is 1/3, meaning 
        that 1/3 of the dataset will be used for testing and the rest for training.
    
    random_state : int, optional
        The seed used by the random number generator for shuffling the data. If `None`, the data 
        will not be shuffled in a reproducible way. Default is `None`.

    Returns:
    -------
    X_train : ndarray
        A numpy array containing the training input features.
    
    y_train : ndarray
        A numpy array containing the training target values.
    
    X_test : ndarray
        A numpy array containing the test input features.
    
    y_test : ndarray
        A numpy array containing the test target values.
    
    """
    data = pd.read_csv(filepath)
    data = data.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Separate the features (X) and target values (y)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values 

    # Calculate the index to split the dataset based on the testsize ratio
    split_index = int(len(data) * (1 - testsize))

    # Split the data into training and test sets
    X_train = X[:split_index]
    y_train = y[:split_index]

    X_test = X[split_index:]
    y_test = y[split_index:]

    # Return the training and test sets
    return X_train, y_train, X_test, y_test

In [ ]:
def constant_attribute(X_train, y_train, X_test, y_test):
    y_mean = np.mean(y_train)

    X_train_ones = np.ones(len(X_train))
    X_test_ones = np.ones(len(X_test))

    y_train_pred = y_mean * X_train_ones
    y_test_pred = y_mean * X_test_ones

    mse_train = np.mean((y_train - y_train_pred)**2)
    mse_test = np.mean((y_test - y_test_pred)**2)

    return mse_train, mse_test

def single_attribute(X_train, y_train, X_test, y_test):
    
    training_errors = []
    test_errors = []

    for i in range(X_train.shape[1]):
        X_train_single = X_train[:, i].reshape(-1, 1)
        X_test_single = X_test[:, i].reshape(-1, 1)

        X_train_augmented = np.hstack([X_train_single, np.ones((X_train_single.shape[0], 1))])
        X_test_augmented = np.hstack([X_test_single, np.ones((X_test_single.shape[0], 1))])

        w = np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train

        y_train_pred = X_train_augmented @ w
        y_test_pred = X_test_augmented @ w

        mse_train = np.mean((y_train - y_train_pred)**2)
        mse_test = np.mean((y_test - y_test_pred)**2)

        training_errors.append(mse_train)
        test_errors.append(mse_test)    

    return training_errors, test_errors

def all_attributes(X_train, y_train, X_test, y_test):
    X_train_augmented = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_test_augmented = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

    w = np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train

    y_train_pred = X_train_augmented @ w
    y_test_pred = X_test_augmented @ w

    mse_train = np.mean((y_train - y_train_pred)**2)
    mse_test = np.mean((y_test - y_test_pred)**2)

    return mse_train, mse_test

In [5]:
def gaussian_kernel(xi, xj, sigma):
    sq_dists = np.sum(xi**2, axis=1)[:, None] + np.sum(xj**2, axis=1)[None, :] - 2 * np.dot(xi, xj.T)
    K = np.exp(-sq_dists / (2 * sigma**2))

    return K

def kernel_ridge_regression(X_train, y_train, gamma, K):
    ell = X_train.shape[0]
    I = np.eye(ell)
    alpha_star = np.linalg.inv(K + gamma * ell * I) @ y_train

    return alpha_star

def cross_validation(X_train, y_train, gamma_values, sigma_values):
    n = len(y_train)
    fold_size = n // 5

    best_gamma = None
    best_sigma = None
    best_mse = np.inf

    mean_mse_vals = np.zeros((len(gamma_values), len(sigma_values)))

    for gamma_index, gamma in enumerate(gamma_values):
        for sigma_index, sigma in enumerate(sigma_values):
            mse_val_errors = []
            
            for i in range(5):
                val_start = i * fold_size
                val_end = (i + 1) * fold_size if i < 4 else n

                # Validation set
                X_val = X_train[val_start:val_end]
                y_val = y_train[val_start:val_end]

                # Training set
                X_train_fold = np.concatenate([X_train[:val_start], X_train[val_end:]], axis=0)
                y_train_fold = np.concatenate([y_train[:val_start], y_train[val_end:]], axis=0)

                K_train = gaussian_kernel(X_train_fold, X_train_fold, sigma)
                K_val = gaussian_kernel(X_val, X_train_fold, sigma)

                # "Training model"
                alpha_star = kernel_ridge_regression(X_train_fold, y_train_fold, gamma, K_train)

                y_val_pred = K_val @ alpha_star

                mse_val_error = np.mean((y_val - y_val_pred) ** 2)
                mse_val_errors.append(mse_val_error)

            mean_mse_vals[gamma_index, sigma_index] = np.mean(mse_val_errors)

            if np.mean(mse_val_errors) < best_mse:
                best_mse = np.mean(mse_val_errors)
                best_gamma = gamma
                best_sigma = sigma

    return best_gamma, best_sigma, best_mse, mean_mse_vals

In [6]:
num_runs = 20
gamma_values = [2**(-40 + i) for i in range(15)]
sigma_values = [2**(7 + i * 0.5) for i in range(13)]

constant_train_errors = []
constant_test_errors = []
single_train_errors = np.zeros((num_runs, 12))
single_test_errors = np.zeros((num_runs, 12))
all_train_errors = []
all_test_errors = []
kernel_train_errors = []
kernel_test_errors = []

for run in range(num_runs):
    X_train, y_train, X_test, y_test = split_dataset(random_state=run)

    mse_train, mse_test = constant_attribute(X_train, y_train, X_test, y_test)
    constant_train_errors.append(mse_train)
    constant_test_errors.append(mse_test)

    training_errors, test_errors = single_attribute(X_train, y_train, X_test, y_test)
    single_train_errors[run] = training_errors
    single_test_errors[run] = test_errors

    mse_train, mse_test = all_attributes(X_train, y_train, X_test, y_test)
    all_train_errors.append(mse_train)
    all_test_errors.append(mse_test)


    # Kernel Ridge Regression
    best_gamma, best_sigma, best_mse, _ = cross_validation(X_train, y_train, gamma_values, sigma_values)

    K_train_best = gaussian_kernel(X_train, X_train, best_sigma)
    K_test_best = gaussian_kernel(X_test, X_train, best_sigma)

    alpha_star_best = kernel_ridge_regression(X_train, y_train, best_gamma, K_train_best)

    y_train_pred = K_train_best @ alpha_star_best
    y_test_pred = K_test_best @ alpha_star_best

    train_mse = np.mean((y_train - y_train_pred) ** 2)
    test_mse = np.mean((y_test - y_test_pred) ** 2)

    kernel_train_errors.append(train_mse)
    kernel_test_errors.append(test_mse)

con_mean_train = np.mean(constant_train_errors)
con_mean_test = np.mean(constant_test_errors)
con_std_train = np.std(constant_train_errors)
con_std_test = np.std(constant_test_errors)

sin_mean_train = np.mean(single_train_errors, axis=0)
sin_mean_test = np.mean(single_test_errors, axis=0)
sin_std_train = np.std(single_train_errors, axis=0)
sin_std_test = np.std(single_test_errors, axis=0)

all_mean_train = np.mean(all_train_errors)
all_mean_test = np.mean(all_test_errors)
all_std_train = np.std(all_train_errors)
all_std_test = np.std(all_test_errors)

ker_mean_train = np.mean(kernel_train_errors)
ker_mean_test = np.mean(kernel_test_errors)
ker_std_train = np.std(kernel_train_errors)
ker_std_test = np.std(kernel_test_errors)

# Take a couple of mins to run


In [7]:
# Display table

methods = [
    "Naive Regression",
    "Linear Regression (attribute 1)",
    "Linear Regression (attribute 2)",
    "Linear Regression (attribute 3)",
    "Linear Regression (attribute 4)",
    "Linear Regression (attribute 5)",
    "Linear Regression (attribute 6)",
    "Linear Regression (attribute 7)",
    "Linear Regression (attribute 8)",
    "Linear Regression (attribute 9)",
    "Linear Regression (attribute 10)",
    "Linear Regression (attribute 11)",
    "Linear Regression (attribute 12)",
    "Linear Regression (all attributes)",
    "Kernel Ridge Regression"
]

data = {
    'Method': methods,
    'MSE train': [
        f"{con_mean_train:.2f} ± {con_std_train:.2f}",
        f"{sin_mean_train[0]:.2f} ± {sin_std_train[0]:.2f}",
        f"{sin_mean_train[1]:.2f} ± {sin_std_train[1]:.2f}",
        f"{sin_mean_train[2]:.2f} ± {sin_std_train[2]:.2f}",
        f"{sin_mean_train[3]:.2f} ± {sin_std_train[3]:.2f}",
        f"{sin_mean_train[4]:.2f} ± {sin_std_train[4]:.2f}",
        f"{sin_mean_train[5]:.2f} ± {sin_std_train[5]:.2f}",
        f"{sin_mean_train[6]:.2f} ± {sin_std_train[6]:.2f}",
        f"{sin_mean_train[7]:.2f} ± {sin_std_train[7]:.2f}",
        f"{sin_mean_train[8]:.2f} ± {sin_std_train[8]:.2f}",
        f"{sin_mean_train[9]:.2f} ± {sin_std_train[9]:.2f}",
        f"{sin_mean_train[10]:.2f} ± {sin_std_train[10]:.2f}",
        f"{sin_mean_train[11]:.2f} ± {sin_std_train[11]:.2f}",
        f"{all_mean_train:.2f} ± {all_std_train:.2f}",
        f"{ker_mean_train:.2f} ± {ker_std_train:.2f}"
    ],
    'MSE test': [
        f"{con_mean_test:.2f} ± {con_std_test:.2f}",
        f"{sin_mean_test[0]:.2f} ± {sin_std_test[0]:.2f}",
        f"{sin_mean_test[1]:.2f} ± {sin_std_test[1]:.2f}",
        f"{sin_mean_test[2]:.2f} ± {sin_std_test[2]:.2f}",
        f"{sin_mean_test[3]:.2f} ± {sin_std_test[3]:.2f}",
        f"{sin_mean_test[4]:.2f} ± {sin_std_test[4]:.2f}",
        f"{sin_mean_test[5]:.2f} ± {sin_std_test[5]:.2f}",
        f"{sin_mean_test[6]:.2f} ± {sin_std_test[6]:.2f}",
        f"{sin_mean_test[7]:.2f} ± {sin_std_test[7]:.2f}",
        f"{sin_mean_test[8]:.2f} ± {sin_std_test[8]:.2f}",
        f"{sin_mean_test[9]:.2f} ± {sin_std_test[9]:.2f}",
        f"{sin_mean_test[10]:.2f} ± {sin_std_test[10]:.2f}",
        f"{sin_mean_test[11]:.2f} ± {sin_std_test[11]:.2f}",
        f"{all_mean_test:.2f} ± {all_std_test:.2f}",
        f"{ker_mean_test:.2f} ± {ker_std_test:.2f}"
    ]
}

df = pd.DataFrame(data)

print(df)

                                Method     MSE train       MSE test
0                     Naive Regression  84.54 ± 5.39  84.47 ± 10.76
1      Linear Regression (attribute 1)  71.25 ± 4.89  73.91 ± 10.40
2      Linear Regression (attribute 2)  74.19 ± 4.34   72.38 ± 8.77
3      Linear Regression (attribute 3)  64.92 ± 4.35   64.56 ± 8.85
4      Linear Regression (attribute 4)  82.02 ± 5.12  82.19 ± 10.62
5      Linear Regression (attribute 5)  69.48 ± 4.45   68.48 ± 8.98
6      Linear Regression (attribute 6)  43.36 ± 3.02   44.46 ± 5.91
7      Linear Regression (attribute 7)  72.69 ± 4.81   72.26 ± 9.68
8      Linear Regression (attribute 8)  79.44 ± 5.26  78.98 ± 10.54
9      Linear Regression (attribute 9)  71.93 ± 4.88   73.08 ± 9.85
10    Linear Regression (attribute 10)  65.59 ± 4.48   66.97 ± 9.02
11    Linear Regression (attribute 11)  62.35 ± 3.99   63.83 ± 8.09
12    Linear Regression (attribute 12)  38.70 ± 2.33   38.50 ± 4.63
13  Linear Regression (all attributes)  22.28 ± 